In [ ]:
import mlflow

# Para que funcione, todos nuestros scripts debemos exportar las siguientes variables de entorno
%env AWS_ACCESS_KEY_ID=minio   
%env AWS_SECRET_ACCESS_KEY=minio123 
%env MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
%env AWS_ENDPOINT_URL_S3=http://localhost:9000

env: AWS_ACCESS_KEY_ID=minio
env: AWS_SECRET_ACCESS_KEY=minio123
env: MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
env: AWS_ENDPOINT_URL_S3=http://localhost:9000


In [2]:
!echo $AWS_ACCESS_KEY_ID
!echo $AWS_SECRET_ACCESS_KEY
!echo $MLFLOW_S3_ENDPOINT_URL

minio
minio123
http://localhost:9000


# MLFlow

## MLFlow Model

### Registrar un modelo

Una vez que hemos subido un modelo a MLFlow, ponerlo en producción es un proceso sencillo. Para ello, nos dirigimos a la pestaña Artifacts del modelo que deseamos implementar. Allí, hacemos clic en el botón *Register Model*, lo que registrará el modelo en el sistema. También podemos hacerlo desde el código, algo que normalmente haríamos en el mismo script o notebook donde entrenamos el modelo.

Primero, creamos el registro:

In [11]:
mlflow.set_tracking_uri('http://localhost:5001')

client = mlflow.MlflowClient()
name = "iris_model_prod"
desc = "This classifier detects if the type of the iris plant"

# Creamos el modelo productivo
registered_model = client.create_registered_model(name=name, description=desc)

### Selección del modelo

Ahora vamos a cargar dos modelos: Regresión Logística y Random Forest. Evaluamos cuál es el mejor y registramos ambos, asignando a uno el tag `champion`, simulando un caso en el que reemplazamos el modelo anterior por uno nuevo.

In [13]:
# Traemos el experimento
experiment_name = "experiment_iris"
experiment = mlflow.get_experiment_by_name(experiment_name)

Primero obtenemos las URIs de los modelos y los descargamos para evaluarlos:

In [19]:
# Traemos la regresión logistica
model_name_lf = 'iris_lr'
run_name_lf = "logistic_regression_exp"
runs_lf_df = mlflow.search_runs(
            experiment_ids=experiment.experiment_id,
            filter_string=f"tags.mlflow.runName = '{run_name_lf}'"   
        )

run_lr_id = runs_lf_df['run_id'].iloc[0]
lr_uri = f"runs:/{run_lr_id}/{model_name_lf}" 

# Cargamos el modelo usando el modulo de scikit-learn
lr_model = mlflow.sklearn.load_model(lr_uri)

In [20]:
# Traemos a random forest
model_name_rf = 'model'
run_name_rf = "random_forest_training"
runs_rf_df = mlflow.search_runs(
            experiment_ids=experiment.experiment_id,
            filter_string=f"tags.mlflow.runName = '{run_name_rf}'"   
        )

run_rf_id = runs_rf_df['run_id'].iloc[0]
rf_uri = f"runs:/{run_rf_id}/{model_name_rf}" 

# Cargamos el modelo usando el modulo de scikit-learn
rf_model = mlflow.sklearn.load_model(rf_uri)

Luego registramos ambos modelos:

In [26]:
result_lr = client.create_model_version(
    name=name,
    source=lr_uri,
    run_id=run_lr_id,
    description="Regresion logistica"
)

2025/07/12 19:48:05 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris_model_prod, version 1


In [27]:
result_rf = client.create_model_version(
    name=name,
    source=rf_uri,
    run_id=run_rf_id,
    description="Random Forest"
)

2025/07/12 19:48:08 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris_model_prod, version 2


A continuación, simulamos el caso en el que el modelo de Random Forest es el actual `champion`:

In [28]:
client.set_registered_model_alias(name, "champion", result_rf.version)

Verificamos que podemos usar este modelo para hacer predicciones:

In [47]:
model_champion = client.get_model_version_by_alias(name, "champion")
model_ml = mlflow.sklearn.load_model(model_champion.source)

print(model_ml.predict([[2.3, 2.2]]))
print(type(model_ml))

[1]
<class 'sklearn.ensemble._forest.RandomForestClassifier'>


Después obtenemos al modelo challenger. Esto se puede hacer de dos maneras:

- Obteniendo directamente la versión del modelo:

In [48]:
challenger_uri = client.get_model_version_download_uri(name, "1")
model_challenger = mlflow.sklearn.load_model(challenger_uri)

print(model_challenger.predict([[2.3, 2.2]]))
print(type(model_challenger))

[0]
<class 'sklearn.linear_model._logistic.LogisticRegression'>


- Asignando un alias (por ejemplo, `challenger`) para facilitar la integración:

In [42]:
client.set_registered_model_alias(name, "challenger", result_lr.version)

In [49]:
model_challenger = client.get_model_version_by_alias(name, "challenger")
model_ch_ml = mlflow.sklearn.load_model(model_challenger.source)

print(model_ch_ml.predict([[2.3, 2.2]]))
print(type(model_ch_ml))

[0]
<class 'sklearn.linear_model._logistic.LogisticRegression'>


Con ambos modelos entrenados, calculamos la accuracy y determinamos cuál fue el mejor:

In [51]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Cargamos los datos
data = load_iris()

# Separamos entre evaluación y testeo
X_train, X_test, y_train, y_test = train_test_split(data['data'][:, :2], data['target'], test_size=0.2, random_state=42)

# Predecimos con los modelos
y_pred_rf = model_ml.predict(X_test)
y_pred_lr = model_ch_ml.predict(X_test)

# Se calculan las métricas
accuracy_rf = accuracy_score(y_test, y_pred_rf)
accuracy_lr = accuracy_score(y_test, y_pred_lr)

print("El accuracy de la Regresión Logistica es:", accuracy_lr)
print("El accuracy de Random Forest es:", accuracy_rf)

if accuracy_lr > accuracy_rf:
    print("El challenger tiene mejores métricas que el modelo de anterior, promocionemos al nuevo champion...")
    client.delete_registered_model_alias(name, "challenger")
    client.set_registered_model_alias(name, "champion", result_lr.version)

    # Este tags nos puede servir si tenemos un bug y debemos hacer un rollback
    client.set_registered_model_alias(name, "old_champion", result_rf.version)


El accuracy de la Regresión Logistica es: 0.9
El accuracy de Random Forest es: 0.7
El challenger tiene mejores métricas que el modelo de anterior, promocionemos al nuevo champion...


Una vez que identificamos al nuevo champion, si lo descargamos, veremos que corresponde al modelo actualizado:

In [52]:
model_champion = client.get_model_version_by_alias(name, "champion")
model_ml = mlflow.sklearn.load_model(model_champion.source)

print(model_ml.predict([[2.3, 2.2]]))
print(type(model_ml))

[0]
<class 'sklearn.linear_model._logistic.LogisticRegression'>


Esta no es la única forma de desplegar modelos. Un segundo enfoque permite el despliegue de un modelo mediante una API REST. Este proceso queda fuera del alcance de este hands-on, pero podés consultar más información en la [documentación oficial de MLFlow](https://mlflow.org/docs/latest/deployment/index.html).

Si querés ver más aplicaciones, podés consultar la notebook del repositorio del proyecto final: [Ejemplo de aplicación](https://github.com/facundolucianna/amq2-service-ml/tree/example_implementation).